# Llama-cpp-python

Create a local Llama2 server, embed documents, and chat with them.

Install RAGStack and additional dependencies.

In [8]:
!pip3 install ragstack-ai sentence_transformers llama-cpp-python 'llama-cpp-python[server]'

Download CodeLlama modeL:

In [9]:
!wget https://huggingface.co/TheBloke/CodeLlama-7B-GGUF/resolve/main/codellama-7b.Q4_0.gguf

--2023-11-20 11:00:42--  https://huggingface.co/TheBloke/CodeLlama-7B-GGUF/resolve/main/codellama-7b.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 18.165.25.13, 18.165.25.36, 18.165.25.68, ...
Connecting to huggingface.co (huggingface.co)|18.165.25.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a1/bc/a1bc01d06501363cc1e896351e68c216246329910db0d64e0b5f6dfa55223d00/33052f6dd41436db2f83bd48017b6fff8ce0184e15a8a227368b4230f1da97b5?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27codellama-7b.Q4_0.gguf%3B+filename%3D%22codellama-7b.Q4_0.gguf%22%3B&Expires=1700755242&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDc1NTI0Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9hMS9iYy9hMWJjMDFkMDY1MDEzNjNjYzFlODk2MzUxZTY4YzIxNjI0NjMyOTkxMGRiMGQ2NGUwYjVmNmRmYTU1MjIzZDAwLzMzMDUyZjZkZDQxNDM2ZGIyZjgzYmQ0ODAxN2I2ZmZmOGNlMDE4NGUxNWE4YTIyNzM2OG

Export the path to your local LLM model.
For more on managing GGUF models, see [](link).

In [ ]:
export MODEL=./codellama-7b.Q4_0.gguf

Start a local Llama2 server.
It will be served at http://localhost:8000/.
n_gpu_layers = 0 to use just CPU.

In [11]:
python3 -m llama_cpp.server --model $MODEL --n_gpu_layers 1

SyntaxError: invalid syntax (2409523322.py, line 1)

Visit http://localhost:8000/docs to see the Swagger UI for your server.